In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
from lxml import etree
import re

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0',
    "Accept-Encoding": "*",
    "Connection": "keep-alive"
    }

In [3]:
main_url = 'https://homecityaruba.com/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0',
    "Accept-Encoding": "*",
    "Connection": "keep-alive"
    }
hyper = ['https://homecityaruba.com/for-sale',
         'https://homecityaruba.com/for-rent'
    ]

In [ ]:
def get_all_url(li):
    all_links = []
    for url in li:
        req = requests.get(url)
        soup = BeautifulSoup(req.content, "html.parser")
        anchors = soup.find('div', {'class': 'preview-properties'})
        links  = anchors.find_all('a', href = True)
        for link in links:
            urls = main_url + link.get('href')
            #
            print(urls)
            all_links.append(urls)
    return all_links
path = get_all_url(hyper)

https://homecityaruba.com//listings/land-bubali-3
https://homecityaruba.com//listings/land-bubali-3
https://homecityaruba.com//listings/land-bubali-3
https://homecityaruba.com//listings/noord-392-m2
https://homecityaruba.com//listings/noord-392-m2
https://homecityaruba.com//listings/noord-392-m2
https://homecityaruba.com//listings/modanza-470-m2
https://homecityaruba.com//listings/modanza-470-m2
https://homecityaruba.com//listings/modanza-470-m2
https://homecityaruba.com//listings/land-bubali-solarium-lot-8
https://homecityaruba.com//listings/land-bubali-solarium-lot-8
https://homecityaruba.com//listings/land-bubali-solarium-lot-8
https://homecityaruba.com//listings/land-bubali-solarium-lot-7
https://homecityaruba.com//listings/land-bubali-solarium-lot-7
https://homecityaruba.com//listings/land-bubali-solarium-lot-7
https://homecityaruba.com//listings/sabana-blanco-lot-4-1203m2
https://homecityaruba.com//listings/sabana-blanco-lot-4-1203m2
https://homecityaruba.com//listings/sabana-bla

In [ ]:
path2 = 